In [1]:
import sys, pdb, os, json, glob
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

# Add the parent directory to PYTHONPATH so that we can use utils.py
sys.path.append('..')
import utils

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
np.random.seed(1337) #stay 1337 AF

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
d = 50 #dimensionality of word vectors
data = utils.build_word_vector_matrix('../data/embeddings/5/vectors_d{}.txt'.format(d))
embeddings, labels, id_to_move, move_to_id = data

In [3]:
X_train, y_train, X_test, y_test, _, _ = utils.load_data('../data/test_moves.txt', embeddings, move_to_id)

unknown move Rgxd2
unknown move Rgxh6


In [4]:
# One-hot encoding
uniq_labels = list(move_to_id.values())

In [5]:
def labels_to_ints(labels):
    ints = []
    for label in labels:
        if label in move_to_id:
            ints.append(move_to_id[label])
        else:
            ints.append(-1)
    return ints

y_train = np.asarray(labels_to_ints(y_train))
y_test  = np.asarray(labels_to_ints(y_test))

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1] * X_test.shape[2]))

In [6]:
y_train = to_categorical(y_train, len(uniq_labels))
y_test = to_categorical(y_test, len(uniq_labels))

In [7]:
hyper = {
    'layers': [{
        'units': 512,
        'init': 'normal',
        'activation': 'relu',
        'dropout': 0.5
        },{
        'units': 512,
        'init': 'normal',
        'activation': 'relu',
        'dropout': 0.5

        },{
        'units': len(uniq_labels),
        'init': 'normal',
        'activation': 'softmax'
    }],
    'loss': 'categorical_crossentropy',
    'optimizer': {
        'type': 'sgd',
        'lr': 0.0001
    },
    'metrics': ['accuracy']

}

model_num = 8

## Create new model, or...
# model, callbacks = utils.create_model(X_train.shape[1], 
#                                       len(uniq_labels), 
#                                       save_dir='../data/models/' + str(model_num),
#                                       hyper=hyper,
#                                       verbose=False)

## Load from checkpoint
model = utils.load_model_from_checkpoint('../data/models/' + str(model_num))

In [8]:
# early stopping
# min_delta = the mininum change in the monitored metric before waiting $patience turns before stopping
# patience = how many consecutive epochs $val_loss change < min_delta before stopping

# callbacks.append(EarlyStopping(monitor='val_loss', 
#                                min_delta=0.001,  
#                                patience=3, 
#                                verbose=1, 
#                                mode='auto'))

In [9]:
# history = model.fit(X_train, 
#                     y_train, 
#                     nb_epoch=10, 
#                     validation_data=(X_test, y_test), 
#                     callbacks=callbacks,
#                     verbose=1)

In [10]:
metrics = model.evaluate(X_test, y_test, verbose=0)
metrics = dict(zip(model.metrics_names, metrics))
for k, v in metrics.items():
    print(k, v)

acc 0.000696378830084
loss 8.22240582681


In [11]:
utils.plot_model_results(history, savedir='./')

NameError: name 'history' is not defined